# Utilities
Enable to create transform the data in usable formats

In [2]:
from custom_functions.gtfs_methods import headways, convert_jsons_to_df, assess_qos_metrics,extract_schedules_from_observed, get_largest_group, turn_hour_to_seconds, compute_EWT, compute_headway
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
pd.set_option('display.max_columns', None)

import logging

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(module)s - : Line (%(lineno)d) - %(message)s",
    level=logging.INFO,
)

/Users/mjdaoudi/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
try:
    headway_t = pd.read_pickle("data/computed/theoretical_schedule.pkl")
    headway_t_qos = pd.read_pickle("data/computed/theoretical_schedule_qos.pkl")
except Exception as e:
    raise e

calendar_dates = pd.read_csv('data/timetables/gtfs3Sept/calendar_dates.txt', sep = ",")
calendar = pd.read_csv('data/timetables/gtfs3Sept/calendar.txt', sep = ",")
trips = pd.read_csv('data/timetables/gtfs3Sept/trips.txt', sep = ",")
stops = pd.read_csv('data/timetables/gtfs3Sept/stops.txt', sep = ",")
stop_times = pd.read_csv('data/timetables/gtfs3Sept/stop_times.txt', sep = ",")
routes = pd.read_csv('data/timetables/gtfs3Sept/routes.txt', sep = ",")

## Theoretical Schedule

In [17]:
path_to_export_theoretical = "data/computed/theoretical_schedule.pkl"

In [ ]:
trips_with_routes = trips.merge(
    routes,
    "left",
    "route_id"
)
stop_times_stop = stop_times.merge(
    stops,
    "left",
    "stop_id"
)
stop_times_with_trips = stop_times_stop.merge(
    trips_with_routes,
    "left",
    "trip_id"
)
headway_theoretical = headways(
    table_times_trips = stop_times_with_trips,
    time_col = "arrival_time",
    stop_col = "stop_id",
    route_col = "route_id",
    direction_col = "direction_id",
    service_col = "service_id"
)
headway_theoretical.to_pickle(path = path_to_export_theoretical)

## Observed Schedule

In [2]:
path_to_read = "/Users/mjdaoudi/Library/CloudStorage/OneDrive-UniversitéLibredeBruxelles/_MS-BDGA.2022-2023/Semester_1/INFO-H423 - Data Mining/STIB-Network-QoS-Analysis/data/vehicule_positions"
path_to_export_observed = "data/computed/observed_schedule.pkl"
path_to_export_observed_filtered = "data/computed/observed_schedule_filtered.pkl"
path_to_export_observed_headways = "data/computed/observed_schedule_dist_0.pkl" 

In [9]:
df = convert_jsons_to_df(folder = path_to_read)
df.to_pickle(path_to_export_observed)

In [5]:
observed_schedule = pd.read_pickle(path_to_export_observed)
transport_type = routes[["route_short_name", "route_type"]].drop_duplicates().reset_index(drop = True)
observed_schedule = observed_schedule.merge(
    transport_type,
    "left",
    "route_short_name"
)

t = observed_schedule.merge(
    stops[["stop_id", "stop_name"]],
    "inner",
    "stop_id"
).rename(columns = {"stop_name" : "current_stop_name"}).merge(
    stops[["stop_id", "stop_name"]],
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id"
).rename(columns = {"stop_name" : "terminus_stop_name", "stop_id_x" : "stop_id"}).drop(columns=["stop_id_y"])

time = t["time"].str.split(":", expand=True)
t["time_seconds"] = (
    time[0].astype(int) * 3600 + time[1].astype(int) * 60 + time[2].astype(int)
)



In [22]:
t

,date,time,route_short_name,distance_from_point,stop_id,stop_id_terminus,route_type,current_stop_name,terminus_stop_name,time_seconds
0,19-09-2021,10:33:18,1,1,8022,8161,1,GARE CENTRALE,STOCKEL,37998
1,19-09-2021,10:38:41,5,0,8022,8262,1,GARE CENTRALE,HERRMANN-DEBROUX,38321
2,19-09-2021,10:39:15,5,0,8022,8262,1,GARE CENTRALE,HERRMANN-DEBROUX,38355
3,19-09-2021,10:40:51,1,0,8022,8161,1,GARE CENTRALE,STOCKEL,38451
4,19-09-2021,10:41:22,1,1,8022,8161,1,GARE CENTRALE,STOCKEL,38482
...,...,...,...,...,...,...,...,...,...,...
15090116,14-09-2021,23:54:28,89,367,1208,2585,3,HENRI REY,JACQUES BREL,86068
15090117,14-09-2021,23:55:00,89,503,1208,2585,3,HENRI REY,JACQUES BREL,86100
15090118,15-09-2021,21:33:25,89,64,1208,2585,3,HENRI REY,JACQUES BREL,77605
15090119,15-09-2021,21:33:56,89,350,1208,2585,3,HENRI REY,JACQUES BREL,77636


In [23]:
t.to_pickle(path_to_export_observed_filtered)

In [24]:
es = pd.read_pickle(path_to_export_observed_filtered)
extracted_schedule = extract_schedules_from_observed(es, "route_type")
extracted_schedule.to_pickle(path_to_export_observed_headways)

In [3]:
extracted_schedule = pd.read_pickle(path_to_export_observed_headways)

### The detection of few stops are causing problems due to some noise in the data

#### Fixed boundaries detection

In [5]:
splt = routes["route_long_name"].str.split(" - ", expand=True)
routes["term1"] = splt[0]
routes["term2"] = splt[1]
terminus = pd.concat([routes["term1"],routes["term2"]]).unique()
terminus

array(["GARE DE L'OUEST", 'SIMONIS', 'ESPLANADE', 'GARE DU NORD',
       'ERASME', 'ROI BAUDOUIN', 'VANDERKINDERE', 'LOUISE',
       'BRUSSELS AIRPORT', 'ETANGS NOIRS', 'HEYSEL', 'BEAULIEU',
       'GROOT-BIJGAARDEN', 'HUNDERENVELD', 'MAES', 'BOONDAEL GARE',
       'GARE DU MIDI', 'LUXEMBOURG', 'BRABANCONNE', 'DE BROUCKERE',
       'DA VINCI', 'DANSAERT', 'PORTE DE NAMUR', 'SCHUMAN', 'ALBERT',
       'GARE CENTRALE', 'MONTGOMERY', 'HEROS', 'VIADUC E40',
       'OBSERVATOIRE', 'ROODEBEEK', 'MOORTEBEEK', 'HEEMBEEK',
       'ANNEESSENS', 'STADE', 'WESTLAND SHOPPING', 'TRONE',
       'VILVOORDE STATION', 'HOPITAL ETTERBEEK-IXELLES', 'AMBIORIX',
       'EUROCONTROL', 'SCHAERBEEK GARE', 'ULB', 'PARKING C',
       'CLEMENCE EVERARD', 'CERIA', 'KRAAINEM', 'HUMANITE',
       'MARIUS RENARD', 'BERCHEM STATION', 'GARE DE BERCHEM', 'MACHTENS',
       'GRAND-PLACE', 'STOCKEL', 'ELISABETH', 'CHURCHILL', 'STALLE (P)',
       'HERRMANN-DEBROUX', 'ARBRE BALLON', 'BRUSSELS CITY', 'UZ-VUB',
       'HEILI

In [93]:
test = get_largest_group(headway_t.groupby(by=["route_id", "stop_id", "service_id", "direction_id"]),35)
test[1].shape, test[0]

((190, 31), (13, '5001F', 237231071, 1))

In [7]:
term = 0
count = 0
count_NA = 0
term_NA = 0
broken_routes = {}
for name, group in extracted_schedule.groupby(by=["date", "route_short_name", "stop_id", "stop_id_terminus"]):
    if group.iloc[0].current_stop_name in terminus:
        group["current_stop_terminus"] = 1
        count +=1 
        if group.shape[0] > 200:
            term += 1
            broken_routes[group.iloc[0].route_short_name] = 1 + broken_routes.get(group.iloc[0].route_short_name, 0)
    else: 
        group["current_stop_terminus"] = 0
        count_NA += 1
        if group.shape[0] > 200:
            term_NA += 1
            broken_routes[group.iloc[0].route_short_name] = 1 + broken_routes.get(group.iloc[0].route_short_name, 0)
            
count, term, count_NA, term_NA, #broken_routes

(12136, 323, 61899, 193)

#### Dynamic boundary detection

It shows that 2% of the data is corrupted with noise.

In [10]:
with open('data/computed/route_bound.json', 'r') as fp:
    route_bound = json.load(fp)

In [13]:
term = 0
count = 0
count_NA = 0
term_NA = 0
broken_routes = {}
for name, group in extracted_schedule.groupby(by=["date", "route_short_name", "stop_id", "stop_id_terminus"]):
    if group.iloc[0].current_stop_name in terminus:
        group["current_stop_terminus"] = 1
        count +=1 
        if group.shape[0] > route_bound[group.iloc[0].route_short_name]:
            term += 1
            broken_routes[group.iloc[0].route_short_name] = 1 + broken_routes.get(group.iloc[0].route_short_name, 0)
    else: 
        group["current_stop_terminus"] = 0
        count_NA += 1
        if group.shape[0] > route_bound[group.iloc[0].route_short_name]:
            term_NA += 1
            broken_routes[group.iloc[0].route_short_name] = 1 + broken_routes.get(group.iloc[0].route_short_name, 0)
            
count, term, count_NA, term_NA, #broken_routes

(12136,
 881,
 61899,
 986,
 {'12': 67,
  '13': 44,
  '14': 41,
  '19': 24,
  '20': 55,
  '25': 4,
  '27': 25,
  '28': 22,
  '3': 33,
  '34': 34,
  '36': 34,
  '38': 53,
  '41': 33,
  '42': 29,
  '45': 18,
  '46': 33,
  '49': 31,
  '54': 53,
  '55': 32,
  '58': 53,
  '59': 66,
  '61': 31,
  '63': 26,
  '64': 44,
  '65': 41,
  '66': 47,
  '71': 96,
  '72': 29,
  '74': 33,
  '75': 26,
  '79': 28,
  '80': 73,
  '86': 49,
  '87': 35,
  '88': 60,
  '92': 31,
  '95': 90,
  '98': 42,
  '21': 7,
  '29': 26,
  '33': 10,
  '37': 11,
  '47': 8,
  '48': 21,
  '50': 21,
  '51': 10,
  '53': 3,
  '56': 18,
  '57': 20,
  '60': 17,
  '69': 11,
  '70': 16,
  '78': 14,
  '8': 2,
  '81': 19,
  '82': 7,
  '83': 10,
  '89': 22,
  '93': 23,
  '17': 2,
  '4': 1,
  '97': 1,
  '77': 1,
  '43': 1})

### Computing Observed Headways

In [28]:
headway_observed = headways(
    table_times_trips = extracted_schedule,
    stop_col = "stop_id",
    route_col = "route_short_name",
    direction_col = "stop_id_terminus",
    service_col = "date",
    time_sec_name = "adjusted_arrival_time(ts)"
)

2022-12-01 23:20:59,939 - root - INFO - gtfs_methods - : Line (64) - Grouping the data
2022-12-01 23:20:59,943 - root - INFO - gtfs_methods - : Line (69) - Computing headways


In [29]:
headway_observed.to_pickle(path = path_to_export_observed_headways)

## QOS metric
Add information regarding the use of regularity or punctuality.

In [3]:
path_to_export_qos = "data/computed/theoretical_schedule_qos.pkl"

In [4]:
qos = assess_qos_metrics(
    timetable =  headway_t,
    stop_col = "stop_id",
    route_col = "route_id",
    direction_col = "direction_id",
    service_col = "service_id")

2022-11-26 22:10:46,052 - root - INFO - gtfs_methods - : Line (266) - Grouping the data
2022-11-26 22:10:46,054 - root - INFO - gtfs_methods - : Line (271) - Computing clusters and assessing regularity


In [6]:
qos.to_pickle(path = path_to_export_qos)

## Cleaning the match

In [2]:
sched = pd.read_pickle("data/computed/schedule_normalized_dist.pkl")
sched_g = sched.groupby(["route_short_name", "stop_id", "direction_id" ,"date_normalized"])

In [5]:
match = pd.read_pickle("data/computed/finally_matches.pkl")
match_g = match.groupby(["route_short_name", "stop_id", "direction_id", "date_normalized"])

In [6]:
rec = []
thresh = 0.85
for name, group in tqdm(sched_g):
    m = match_g.get_group(name).dropna(subset="adjusted_arrival_time")
    if m.shape[0] >= thresh * group.shape[0]:
        rec.extend(m.to_dict("records"))

100%|██████████| 45524/45524 [06:00<00:00, 126.24it/s] 


In [7]:
clean = pd.DataFrame(rec)

In [9]:
clean.to_pickle("data/computed/finally_matches_cleaned.pkl")

## Excess Waiting Time

In [3]:
res = pd.read_pickle("data/computed/finally_matches_cleaned.pkl")

Drop Columns and computing seconds

In [4]:
with open("data/computed/useless_col.json",'r') as c :
    col_to_drop = json.loads(c.read())
res = res.drop(columns= col_to_drop)

res = turn_hour_to_seconds("departure_time", res, "time_seconds")
res = turn_hour_to_seconds("adjusted_arrival_time", res, "adjusted_arrival_time(ts)")

res= res.rename(columns= {"departure_time":"theoretical_time", "time_seconds": "th_time_sec", "adjusted_arrival_time": "adj_real_time", "adjusted_arrival_time(ts)": "real_time_sec", "headway_min" : "headway_th"})

Computing the real Headway

In [6]:
res = res.groupby(by= ["stop_id", "route_id", "direction_id", "service_id" ,"date_normalized"], as_index=False).apply(lambda x: compute_headway(x.sort_values("th_time_sec"), headway_col_name= "headway_real", time_sec_col_name="real_time_sec"))

In [7]:
res = res.reset_index().drop(columns=["level_0", "level_1"])
res

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333


Computing the EWT

In [8]:

def compute_EWT(
    timetable: pd.DataFrame,
    stop_col: str = "stop_id",
    route_col: str = "route_id",
    direction_col: str = "direction_id",
    service_col: str = "service_id",
    date_col: str = "date_normalized",
    cluster_col: str = "clusters",
    s_headway_col: str = "headway_th",
    a_headway_col: str = "headway_real",
) -> pd.DataFrame:

    logging.info("Computing the square of headways")
    timetable["s_headway_square"] = timetable[s_headway_col] ** 2
    timetable["a_headway_square"] = timetable[a_headway_col] ** 2

    logging.info("Computing SWT & AWT per stop, route, direction, date and cluster")
    SWT = timetable.groupby(
        by=[stop_col, route_col, direction_col, service_col, date_col, cluster_col]
    ).apply(lambda x: x["s_headway_square"].sum() / (2 * (x[s_headway_col].sum())))
    AWT = timetable.groupby(
        by=[stop_col, route_col, direction_col, service_col, date_col, cluster_col]
    ).apply(lambda x: x["a_headway_square"].sum() / (2 * (x[a_headway_col].sum())))

    SWT.name = "SWT"
    AWT.name = "AWT"

    timetable = timetable.merge(
        SWT,
        on=[stop_col, route_col, direction_col, service_col, date_col, cluster_col],
        how="left",
    ).merge(
        AWT,
        on=[stop_col, route_col, direction_col, service_col, date_col, cluster_col],
        how="left",
    )

    logging.info("Computing EWT")
    timetable["EWT"] = timetable["AWT"] - timetable["SWT"]

    timetable = timetable.drop(columns=["s_headway_square", "a_headway_square"])

    return timetable

res = compute_EWT(
    res,
    s_headway_col = "headway_th"
    )
res["EWT_label"] = np.where(res["EWT"] >= 1, "irregular", "regular")

2022-12-16 18:55:19,288 - root - INFO - 3684019190 - : Line (13) - Computing the square of headways
2022-12-16 18:55:19,320 - root - INFO - 3684019190 - : Line (17) - Computing SWT & AWT per stop, route, direction, date and cluster
/var/folders/dz/p7gp3f295wxf90tn1dv3qkq40000gn/T/ipykernel_12712/3684019190.py:20: RuntimeWarning: invalid value encountered in double_scalars
  ).apply(lambda x: x["s_headway_square"].sum() / (2 * (x[s_headway_col].sum())))
/var/folders/dz/p7gp3f295wxf90tn1dv3qkq40000gn/T/ipykernel_12712/3684019190.py:23: RuntimeWarning: invalid value encountered in double_scalars
  ).apply(lambda x: x["a_headway_square"].sum() / (2 * (x[a_headway_col].sum())))
2022-12-16 18:56:01,742 - root - INFO - 3684019190 - : Line (38) - Computing EWT


In [10]:
res.to_pickle("data/computed/final_matched_clean_EWT.pkl")

## Computing delays/advances

In [13]:
res = pd.read_pickle("data/computed/final_matched_clean_EWT.pkl")

In [14]:
res["delay"] = res["real_time_sec"] - res["th_time_sec"]

res["minutes_schedule"] = res["theoretical_time"].apply(lambda x : x.split(":")[1]).astype("int")
res["minutes_observations"] = res["adj_real_time"].apply(lambda x : x.split(":")[1]).astype("int")
res["hours_schedule"] = res["theoretical_time"].apply(lambda x : x.split(":")[0]).astype("int")
res["hours_observations"] = res["adj_real_time"].apply(lambda x : x.split(":")[0]).astype("int")

res["hours_observations"]= np.where(
    res["hours_observations"].isin([0,1]), 
    res["hours_observations"] + 24, 
    res["hours_observations"]
    
    )

res["delay_label_strict"] = np.where(
    (res["delay"] < 0) & ((res["minutes_observations"] < res["minutes_schedule"]) | (res["hours_observations"] < res["hours_schedule"])),
    "adv",
    np.where(
        (res["delay"] > 0) & ((res["minutes_observations"] > res["minutes_schedule"]) | (res["hours_observations"] > res["hours_schedule"])),
        "lat",
        np.NaN
    )
)

res["delay_label_60"] = np.where(
    (res["delay"] < -60),
    "adv",
    np.where(
        (res["delay"] > 60),
        "lat",
        np.NaN
    )
)

res["delay_label_120"] = np.where(
    (res["delay"] < -120),
    "adv",
    np.where(
        (res["delay"] > 120),
        "lat",
        np.NaN
    )
)

res = res.drop(columns=["minutes_schedule", "minutes_observations", "hours_schedule", "hours_observations"])
res

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN,8.654762,11.624663,2.969901,irregular,0,nan,nan,nan
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667,8.654762,11.624663,2.969901,irregular,796,lat,lat,lat
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000,8.654762,11.624663,2.969901,irregular,-29,adv,nan,nan
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000,8.654762,11.624663,2.969901,irregular,4,nan,nan,nan
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333,8.654762,11.624663,2.969901,irregular,18,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333,6.382353,7.235564,0.853211,regular,-82,adv,adv,nan
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000,6.382353,7.235564,0.853211,regular,-1,adv,nan,nan
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667,6.382353,7.235564,0.853211,regular,525,lat,lat,lat
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333,6.382353,7.235564,0.853211,regular,86,lat,lat,nan


In [15]:
res.to_pickle("data/computed/matches_clean_EWT_delay.pkl")

## Adding utilities

In [9]:
res = pd.read_pickle("data/computed/matches_clean_EWT_delay.pkl")

In [6]:
res["hour"] = res.theoretical_time.apply(
    lambda x: int(x.split(":")[0])
)
res["day"] = res.date_normalized.apply(
    lambda x: int(x.split("-")[0])
)

In [7]:
res.to_pickle("data/computed/matches_clean_EWT_delay.pkl")

## Subsampling the data set

In [26]:
res = pd.read_pickle("data/computed/matches_EWT_delay.pkl")

In [27]:
res_g = res.groupby(by=["route_short_name", "stop_id"])
thresh = 4
g = []
i = 0
for name, group in tqdm(res_g):
    if i <= thresh:
        g.append(group)
    i += 1

sample = pd.concat(g)

sample

100%|██████████| 3386/3386 [00:02<00:00, 1139.22it/s]


,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,delay,delay_label
3755153,112949632236270000,23:08:53,8011,14,DE BROUCKERE,1,236270000,ERASME,1,1,ERASME - HERRMANN-DEBROUX,83333,10.000000,cluster_4,10.00,1,2021-09-01,2021-09-17,06-09-2021,588.817544,0.0,8011,8642,ERASME,8642,workdays,1,M,83586,23:13:06,110765,NaN,10.866667,5.066431,6.599104,1.532673,253,lat
3755159,112949813236270000,05:09:07,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,18547,NaN,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,18476,05:07:56,111546,NaN,NaN,2.796185,8.242382,5.446197,-71,adv
3755163,112950843236270000,06:07:09,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22029,2.483333,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22304,06:11:44,113846,1.0,0.733333,2.796185,8.242382,5.446197,275,lat
3755164,112950421236270000,06:12:25,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,22345,5.266667,cluster_0,4.75,1,2021-09-01,2021-09-17,07-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,22260,06:11:00,113386,NaN,17.200000,2.796185,8.242382,5.446197,-85,adv
3755336,112952547236270000,05:56:26,8011,11,DE BROUCKERE,1,236270000,GARE DE L'OUEST,1,1,ERASME - HERRMANN-DEBROUX,21386,3.183333,cluster_0,4.75,1,2021-09-01,2021-09-17,08-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,workdays,1,M,21310,05:55:10,113907,NaN,10.933333,2.531103,4.595001,2.063899,-76,adv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3773839,112949518236269500,23:21:38,8031,14,PARC,2,236269500,ERASME,1,1,GARE DE L'OUEST - STOCKEL,84098,10.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,0.0,8031,8642,ERASME,8642,saturday,1,M,84327,23:25:27,497,NaN,11.166667,5.745127,5.820706,0.075579,229,lat
3773840,112947678236269500,23:31:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,84698,10.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,0.0,8031,8731,GARE DE L'OUEST,8731,saturday,1,M,84700,23:31:40,307,NaN,6.216667,5.745127,5.820706,0.075579,2,nan
3773841,112947668236269500,23:46:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,85598,15.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,0.0,8031,8731,GARE DE L'OUEST,8731,saturday,1,M,85726,23:48:46,287,NaN,17.100000,5.745127,5.820706,0.075579,128,lat
3773842,112947687236269500,24:06:38,8031,14,PARC,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,86798,20.000000,cluster_3,10.00,1,2021-09-04,2021-09-18,18-09-2021,471.506564,0.0,8031,8731,GARE DE L'OUEST,8731,sunday,1,M,86796,00:06:36,325,NaN,17.833333,5.745127,5.820706,0.075579,-2,nan


In [28]:
sample_name = "matches_EWT_delay_subsampled"
sample.to_pickle("data/computed/" + sample_name + ".pkl")

## Service matching

In [11]:
headways = pd.read_pickle(path_to_export_observed_headways)

In [54]:
headways

,date,time,route_short_name,distance_from_point,stop_id,stop_id_terminus,time_seconds,headway_min
0,07-09-2021,12:14:33,13,0,1008,4111,44073,NaN
1,07-09-2021,12:15:06,13,0,1008,4111,44106,0.550000
2,07-09-2021,12:15:37,13,0,1008,4111,44137,0.516667
3,07-09-2021,12:16:09,13,0,1008,4111,44169,0.533333
4,07-09-2021,12:16:41,13,0,1008,4111,44201,0.533333
...,...,...,...,...,...,...,...,...
6886233,20-09-2021,18:28:37,61,0,9996,9996,66517,1091.133333
6886234,20-09-2021,18:29:07,61,0,9996,9996,66547,0.500000
6886235,20-09-2021,18:29:37,61,0,9996,9996,66577,0.500000
6886236,20-09-2021,18:30:10,61,0,9996,9996,66610,0.550000


In [43]:
headways_stop = headways.merge(
    stops,
    "left",
    left_on = "stop_id_terminus",
    right_on = "stop_id"
)

In [42]:
headways_stop

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,time_seconds,headway_min,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,07-09-2021,12:14:33,13,0,1008,4111,44073,NaN,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN,20,ETANGS NOIRS - UZ-VUB,NaN,3,NaN,91BEE7,000000
1,07-09-2021,12:15:06,13,0,1008,4111,44106,0.550000,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN,20,ETANGS NOIRS - UZ-VUB,NaN,3,NaN,91BEE7,000000
2,07-09-2021,12:15:37,13,0,1008,4111,44137,0.516667,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN,20,ETANGS NOIRS - UZ-VUB,NaN,3,NaN,91BEE7,000000
3,07-09-2021,12:16:09,13,0,1008,4111,44169,0.533333,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN,20,ETANGS NOIRS - UZ-VUB,NaN,3,NaN,91BEE7,000000
4,07-09-2021,12:16:41,13,0,1008,4111,44201,0.533333,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN,20,ETANGS NOIRS - UZ-VUB,NaN,3,NaN,91BEE7,000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7394185,20-09-2021,18:28:37,61,0,9996,9996,66517,1091.133333,9996,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0.0,NaN,59,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000
7394186,20-09-2021,18:29:07,61,0,9996,9996,66547,0.500000,9996,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0.0,NaN,59,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000
7394187,20-09-2021,18:29:37,61,0,9996,9996,66577,0.500000,9996,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0.0,NaN,59,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000
7394188,20-09-2021,18:30:10,61,0,9996,9996,66610,0.550000,9996,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0.0,NaN,59,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000


In [40]:
routes

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,2,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF
1,4,2,SIMONIS - ELISABETH,NaN,1,NaN,F57000,FFFFFF
2,5,3,ESPLANADE - CHURCHILL,NaN,0,NaN,B5BA05,000000
3,10,4,GARE DU NORD - STALLE (P),NaN,0,NaN,F25482,000000
4,1,5,ERASME - HERRMANN-DEBROUX,NaN,1,NaN,E6B012,FFFFFF
...,...,...,...,...,...,...,...,...
93,84,N12,GARE CENTRALE - STALLE P,NaN,3,NaN,ED7807,000000
94,86,N13,GARE CENTRALE - WESTLAND SHOPPING,NaN,3,NaN,A12944,000000
95,92,N16,GARE CENTRALE - BERCHEM STATION,NaN,3,NaN,80C29C,FFFFFF
96,87,N16,GARE CENTRALE - GARE DE BERCHEM,NaN,3,NaN,74C095,000000


In [13]:
("07-09-2021", 13, 1008, 4111)

('07-09-2021', 13, 1008, 4111)

In [33]:
headways_stop[
    (headways_stop.date == "07-09-2021") &
    (headways_stop.route_short_name == "13") &
    (headways_stop.stop_id_x == "1008") &
    (headways_stop.stop_id_terminus == "4111")
]

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,time_seconds,headway_min,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,07-09-2021,12:14:33,13,0,1008,4111,44073,NaN,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
1,07-09-2021,12:15:06,13,0,1008,4111,44106,0.550000,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
2,07-09-2021,12:15:37,13,0,1008,4111,44137,0.516667,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
3,07-09-2021,12:16:09,13,0,1008,4111,44169,0.533333,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
4,07-09-2021,12:16:41,13,0,1008,4111,44201,0.533333,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
5,07-09-2021,19:30:23,13,0,1008,4111,70223,433.700000,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN
6,07-09-2021,19:30:55,13,0,1008,4111,70255,0.533333,4111,NaN,ETANGS NOIRS,NaN,50.857072,4.332195,NaN,NaN,0.0,NaN


In [22]:
calendar

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,234578052,1,1,1,1,1,0,0,20210823,20210831
1,236986502,0,0,0,0,0,1,0,20210828,20210828
2,236988602,0,0,0,0,0,0,1,20210829,20210829
3,236270000,1,1,1,1,1,0,0,20210901,20210917
4,236269500,0,0,0,0,0,1,0,20210904,20210918
...,...,...,...,...,...,...,...,...,...,...
432,237102501,0,0,0,0,0,1,0,20210911,20210911
433,237472602,0,0,0,0,0,0,1,20210912,20210912
434,237451600,0,0,0,0,0,0,1,20210905,20210905
435,238036502,0,0,0,0,0,1,0,20210918,20210918


In [21]:
headways_theoretical = pd.read_pickle(path_to_export_theoretical)

In [14]:
headways_theoretical[headways_theoretical.stop_id == "1008"].info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1038 entries, 108018 to 109055
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trip_id           1038 non-null   int64  
 1   arrival_time      1038 non-null   object 
 2   departure_time    1038 non-null   object 
 3   stop_id           1038 non-null   object 
 4   stop_sequence     1038 non-null   int64  
 5   pickup_type       1038 non-null   int64  
 6   drop_off_type     1038 non-null   int64  
 7   stop_code         0 non-null      float64
 8   stop_name         1038 non-null   object 
 9   stop_desc         0 non-null      float64
 10  stop_lat          1038 non-null   float64
 11  stop_lon          1038 non-null   float64
 12  zone_id           0 non-null      float64
 13  stop_url          0 non-null      float64
 14  location_type     1038 non-null   int64  
 15  parent_station    0 non-null      float64
 16  route_id          1038 non-null   i

In [15]:
trips_date = trips.merge(
    routes,
    "left",
    "route_id"
).merge(
    calendar,
    "left",
    "service_id"
)

In [16]:
trips_date

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,24,235954071,112387248235954071,SIMONIS,1,8902800,087b0196,87,SIMONIS - BEEKKANT,NaN,3,NaN,4C8B33,000000,1,1,1,1,1,0,0,20210823,20210831
1,24,235954071,112387249235954071,SIMONIS,1,8902802,087b0196,87,SIMONIS - BEEKKANT,NaN,3,NaN,4C8B33,000000,1,1,1,1,1,0,0,20210823,20210831
2,24,235954071,112387253235954071,SIMONIS,1,8902801,087b0196,87,SIMONIS - BEEKKANT,NaN,3,NaN,4C8B33,000000,1,1,1,1,1,0,0,20210823,20210831
3,24,235954071,112387254235954071,SIMONIS,1,8902800,087b0196,87,SIMONIS - BEEKKANT,NaN,3,NaN,4C8B33,000000,1,1,1,1,1,0,0,20210823,20210831
4,24,235954071,112387255235954071,SIMONIS,1,8902802,087b0196,87,SIMONIS - BEEKKANT,NaN,3,NaN,4C8B33,000000,1,1,1,1,1,0,0,20210823,20210831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135244,8,236519600,113028497236519600,DIEWEG,0,8919035,097t0096,97,LOUISE - DIEWEG,NaN,0,NaN,A12944,000000,0,0,0,0,0,0,1,20210912,20210912
135245,8,236519600,113028628236519600,DIEWEG,0,8919038,097t0096,97,LOUISE - DIEWEG,NaN,0,NaN,A12944,000000,0,0,0,0,0,0,1,20210912,20210912
135246,8,236519600,113028637236519600,DIEWEG,0,8919037,097t0102,97,LOUISE - DIEWEG,NaN,0,NaN,A12944,000000,0,0,0,0,0,0,1,20210912,20210912
135247,8,236519600,113028638236519600,DIEWEG,0,8919036,097t0102,97,LOUISE - DIEWEG,NaN,0,NaN,A12944,000000,0,0,0,0,0,0,1,20210912,20210912


In [19]:
trips_date["start_date"] = pd.to_datetime(trips_date.start_date, format='%Y%m%d')
trips_date["end_date"] = pd.to_datetime(trips_date.end_date, format='%Y%m%d')

In [34]:
headways_group = headways_stop.groupby(by=["date", "route_short_name", "stop_name"])

In [36]:
for names, group in headways_group:
    print(names)

('06-09-2021', '1', 'DELTA')
('06-09-2021', '1', 'ERASME')
('06-09-2021', '1', "GARE DE L'OUEST")
('06-09-2021', '1', 'HERRMANN-DEBROUX')
('06-09-2021', '1', 'MERODE')
('06-09-2021', '1', 'STOCKEL')
('06-09-2021', '12', 'MEISER')
('06-09-2021', '12', 'TRONE')
('06-09-2021', '13', 'ETANGS NOIRS')
('06-09-2021', '13', 'RIETHUISEN')
('06-09-2021', '13', 'UZ-VUB')
('06-09-2021', '14', 'GARE DU NORD')
('06-09-2021', '14', 'HEYSEL')
('06-09-2021', '14', 'UZ BRUSSEL')
('06-09-2021', '17', 'BEAULIEU')
('06-09-2021', '17', 'HEILIGENBORRE')
('06-09-2021', '19', 'GUILL. DE GREEF')
('06-09-2021', '19', 'HUNDERENVELD')
('06-09-2021', '19', 'SCHWEITZER')
('06-09-2021', '2', 'DELACROIX')
('06-09-2021', '2', 'ELISABETH')
('06-09-2021', '2', "GARE DE L'OUEST")
('06-09-2021', '2', 'ROI BAUDOUIN')
('06-09-2021', '2', 'SIMONIS')
('06-09-2021', '20', 'GARE DU NORD')
('06-09-2021', '20', 'HUNDERENVELD')
('06-09-2021', '20', 'OSSEGHEM')
('06-09-2021', '21', 'BORDET STATION')
('06-09-2021', '21', 'MAES')
('06

### Duplicates issue

In [18]:
routes.route_short_name.unique().shape

(92,)

In [19]:
routes.route_id.unique().shape

(98,)

In [20]:
routes.route_short_name.unique().shape

(92,)

In [21]:
routes[routes.duplicated(subset=['route_short_name'])]

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
18,27,27,LUXEMBOURG - PLEIADES,NaN,3,NaN,91BEE7,000000
68,16,82,GARE DE BERCHEM - DROGENBOS,NaN,0,NaN,91BEE7,000000
70,73,83,GARE DE BERCHEM - VAL MARIA,NaN,3,NaN,B4BD10,000000
72,89,86,MACHTENS - GARE CENTRALE,NaN,3,NaN,338C26,FFFFFF
76,33,89,WESTLAND SHOPPING - GARE CENTRALE,NaN,3,NaN,B4BD10,000000
96,87,N16,GARE CENTRALE - GARE DE BERCHEM,NaN,3,NaN,74C095,000000


###  Next of code

There is a problem in the service IDs allocation with 17.5% (38% without the headsign matching) of cases. A direct match with the date, weekday, route and headsign cannot be done in all cases. A solution could be to randomly allocate the service_ids. Anyway they are applied on the same periods, for the same trips. It would affect the result  slightly. Another solution can simply be to put NaN and don't use them as such first.

In [47]:
issues = 0
cases = 0
for names, group in headways_group:
    date = pd.to_datetime(names[0], format="%d-%m-%Y")
    date_weekday = date.strftime("%A").lower()
    route_short_name_used = names[1]
    headsign = names[2]

    filter = trips_date[
        (trips_date.route_short_name == route_short_name_used) &
        (trips_date.start_date <= date) &
        (date <= trips_date.end_date) &
        (trips_date[date_weekday] == 1) &
        (trips_date.trip_headsign  == headsign)]
    """
    calendar_to_check = calendar_dates[calendar_dates.date.isin([date])]
    if len(filter.service_id.unique()) >= 2:
        for service in filter.service_id.unique():
            if service in calendar_to_check["service_id"].values:
                filter.service_id = service
            else:
                pass
    """
    if len(filter.service_id.unique()) != 1:
        #print(names, filter.service_id.unique())
        #print(filter)
        issues += 1
    cases += 1

print(round((issues/cases)*100, 2))

17.44


NaN solution

In [53]:
hg = []
issues = 0
cases = 0
for names, group in headways_group:
    date = pd.to_datetime(names[0], format="%d-%m-%Y")
    date_weekday = date.strftime("%A").lower()
    route_short_name_used = names[1]
    headsign = names[2]

    filter = trips_date[
        (trips_date.route_short_name == route_short_name_used) &
        (trips_date.start_date <= date) &
        (date <= trips_date.end_date) &
        (trips_date[date_weekday] == 1) &
        (trips_date.trip_headsign  == headsign)]

    """ What for ?
    calendar_to_check = calendar_dates[calendar_dates.date.isin([date])]
    if len(filter.service_id.unique()) >= 2:
        for service in filter.service_id.unique():
            if service in calendar_to_check["service_id"].values:
                filter.service_id = service
            else:
                pass
    """

    if len(filter.service_id.unique()) != 1:
        #service_id_matched = np.NaN
        issues += 1
    else: 
        #service_id_matched = filter.service_id.unique()[0]
        pass

    #group["service_id_matched"] = service_id_matched
    #hg.append(group.to_dict('records'))
    cases += 1

#headways_service_ids = pd.DataFrame(hg)
print(round((issues/cases)*100, 2))

58.93


### Check what's happening with the service_id

In [24]:
calendar[calendar.service_id.isin([236963001, 236965001, 237231071])]

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
52,236963001,1,1,1,1,1,0,0,20210901,20210917
78,237231071,1,1,1,1,1,0,0,20210901,20210915
79,236965001,1,1,1,1,1,0,0,20210902,20210917


In [25]:
calendar_dates[calendar_dates.service_id.isin([236963001, 236965001, 237231071])]

,service_id,date,exception_type
13,237231071,20210902,2
14,237231071,20210903,2
15,237231071,20210906,2
16,237231071,20210907,2
17,237231071,20210909,2
18,237231071,20210910,2
19,237231071,20210913,2
20,237231071,20210914,2
21,236965001,20210908,2
22,236965001,20210915,2


In [39]:
trips_date[trips_date.service_id.isin([236963001, 236965001, 237231071]) & trips_date.route_short_name.isin(["19"]) & trips_date.trip_headsign.isin(["GUILL. DE GREEF"])]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
116938,13,236963001,113191670236963001,GUILL. DE GREEF,0,8927354,019t0104,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-01,2021-09-17
116939,13,236963001,113191671236963001,GUILL. DE GREEF,0,8927357,019t0104,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-01,2021-09-17
116942,13,236963001,113191773236963001,GUILL. DE GREEF,0,8927356,019t0104,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-01,2021-09-17
116943,13,236963001,113191774236963001,GUILL. DE GREEF,0,8927359,019t0104,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-01,2021-09-17
116946,13,236963001,113203343236963001,GUILL. DE GREEF,0,8927353,019t0104,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-01,2021-09-17
117507,13,236965001,113179758236965001,GUILL. DE GREEF,1,8927477,019t0359,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-02,2021-09-17
117804,13,236965001,113180080236965001,GUILL. DE GREEF,1,8927498,019t0359,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-02,2021-09-17
117805,13,236965001,113180081236965001,GUILL. DE GREEF,1,8927471,019t0359,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-02,2021-09-17
117806,13,236965001,113180082236965001,GUILL. DE GREEF,1,8927482,019t0359,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-02,2021-09-17
117807,13,236965001,113180083236965001,GUILL. DE GREEF,1,8927473,019t0359,19,GROOT-BIJGAARDEN - DE WAND,NaN,0,NaN,DE3B21,FFFFFF,1,1,1,1,1,0,0,2021-09-02,2021-09-17


In [44]:
headways_stop[(headways_stop.route_short_name == "19") & (headways_stop.date == "06-09-2021") & (headways_stop.stop_name == "GUILL. DE GREEF")]

,date,time,route_short_name,distance_from_point,stop_id_x,stop_id_terminus,time_seconds,headway_min,stop_id_y,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
2177633,06-09-2021,09:59:31,19,0,2637,5010,35971,NaN,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
2177634,06-09-2021,18:33:53,19,0,2637,5010,66833,514.366667,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
2177635,06-09-2021,19:08:56,19,0,2637,5010,68936,35.050000,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
4110811,06-09-2021,10:00:34,19,0,4955,5010,36034,NaN,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
4110812,06-09-2021,10:01:08,19,0,4955,5010,36068,0.566667,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113920,06-09-2021,21:59:50,19,0,5001,5010,79190,0.500000,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
4113921,06-09-2021,22:00:20,19,0,5001,5010,79220,0.500000,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
4113922,06-09-2021,22:00:51,19,0,5001,5010,79251,0.516667,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
4161174,06-09-2021,18:36:30,19,0,5003,5010,66990,NaN,5010,NaN,GUILL. DE GREEF,NaN,50.882189,4.335515,NaN,NaN,0.0,NaN
